<a href="https://colab.research.google.com/github/programminghistorian/jekyll/blob/gh-pages/assets/TEST-colab-notebook/TEST-colab-notebook.ipynb\" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preliminaries

In [ ]:
%%capture
%mkdir yearbook
%cd yearbook
!pip install --upgrade --no-cache-dir gdown
!gdown --id "1NHT8NN8ClBEnUC5VqkP3wr2KhyiIQzyU"
!unzip PHfiles.zip
%mkdir images
!pip install PyMuPDF
!pip install dlib
!pip install DeepFace
import os, shutil, fitz, cv2, numpy as np, pandas as pd, dlib, tensorflow as tf
from os.path import dirname, join
from deepface import DeepFace

##PDF Conversion

In [ ]:
path = r'./'
pdfs = [f for f in os.listdir(path) if f.endswith('.pdf')]
for pdf in pdfs:
    os.chdir(os.path.join('./images'))
    os.mkdir((pdf.split(".")[0]))
    newdir = (os.path.join('./images/' + os.path.join(pdf.split(".")[0])))
    os.chdir("..")
    print ("Now copying images into " + (newdir))
    shutil.copy(pdf, newdir)
    os.chdir(newdir)
    doc = fitz.open(pdf)
    for page in doc:
      pix = page.get_pixmap()
      pix.save("page-%i.png" % page.number)
    os.chdir(os.path.dirname(os.getcwd()))
    os.chdir("..")



Now copying images into ./images/1921
Now copying images into ./images/1961
Now copying images into ./images/1931
Now copying images into ./images/1941
Now copying images into ./images/1951
Now copying images into ./images/1911


##Object Detection and Facial Recognition: Code

In [ ]:
path = r'./'

os.chdir(os.path.join(path + 'images'))
dirs = os.listdir(path)
for dir in dirs:
    os.chdir(os.path.join(path + dir))
    pngs = [f for f in os.listdir(path) if f.endswith('.png')]

    if not os.path.exists((dir) + ' faces'):
        print("New 'faces' directory created in " + (dir) + " folder")
        os.makedirs((dir) + ' faces')

        count = 0
        for png in pngs:
            image = cv2.imread(png)

            greyscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

            detected_faces = face_cascade.detectMultiScale(image=greyscale_image, scaleFactor=1.9, minNeighbors=4)

            count = 0
            for (x,y,w,h) in detected_faces:
                try:
                    xpadding = 20
                    ypadding = 40
                    crop_face = image[y-ypadding: y + h+ypadding, x-xpadding: x + w+xpadding]
                    count+=1
                    face = cv2.rectangle(crop_face,(x,y),(x+w,y+h),(255,0,0),2)
                    cv2.imwrite(path + (dir) + ' faces/' + str(count) + '_' + png, face)
                except (Exception):
                    print("An error happened")
                    continue
            os.remove(os.path.join(path, png))
    os.chdir("..")


New 'faces' directory created in 1961 folder
New 'faces' directory created in 1931 folder
New 'faces' directory created in 1911 folder
New 'faces' directory created in 1921 folder
New 'faces' directory created in 1951 folder
New 'faces' directory created in 1941 folder


##Identify Smiles: Code

In [ ]:
%cd ..
number_smiles = 0
smile_counts = []
number_nonsmiles = 0
nonsmile_counts = []
num_errors = 0
error_counts = []
pngs = []

file_count = 0
file_count_list = []

years = ['1911', '1921', '1931', '1941', '1951', '1961']

for year in years:
    path = r'./images' + '/' + year
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            path = path + '/' + (year + ' faces')
            if(file_count != 0):
                file_count_list.append(file_count)
            file_count = 0
            for f in os.listdir(path):
                if f.endswith('.png'):
                    pngs.append(path + '/' + f)
                    file_count = file_count + 1

file_count_list.append(file_count)

total_loops = 0
count = 0
iterator = 0
for png in pngs:
    try:
        total_loops = total_loops + 1
        count = count + 1
        if(count != (file_count_list[iterator] + 1)):
            demography = DeepFace.analyze(png, actions = ['emotion'])
            print(demography)
            if(demography[0]['dominant_emotion'] == 'happy'):
                number_smiles = number_smiles + 1
            else:
                number_nonsmiles = number_nonsmiles + 1

        else:
            count = count - 1
            smile_counts.append(number_smiles / count)
            nonsmile_counts.append(number_nonsmiles / count)
            error_counts.append(num_errors / count)
            number_smiles = 0
            number_nonsmiles = 0
            num_errors = 0
            iterator = iterator + 1
            count = 0

    except (Exception):
        num_errors = num_errors + 1
        print("An error happened")
        continue

smile_counts.append(number_smiles / count)
nonsmile_counts.append(number_nonsmiles / count)
error_counts.append(num_errors / count)

dict = {'Years': years, 'Smiles': smile_counts, 'Non-Smiles': nonsmile_counts, "Error Weight": error_counts}
data = pd.DataFrame(dict)
data.to_csv('YearbookOutput.csv', index=False)
print(count)


/content/yearbook
24-01-31 16:15:20 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 18.6MB/s]


[{'emotion': {'angry': 2.1246447489089073, 'disgust': 1.901664848713893e-08, 'fear': 0.034286557916850396, 'happy': 0.0002512492229182152, 'sad': 4.2414514879642145, 'surprise': 1.7301296109425963e-07, 'neutral': 93.59936194427468}, 'dominant_emotion': 'neutral', 'region': {'x': 28, 'y': 51, 'w': 63, 'h': 63}, 'face_confidence': 10.072929168702103}]
[{'emotion': {'angry': 2.7204811573028564, 'disgust': 3.8496024146184027e-07, 'fear': 1.0063357651233673, 'happy': 0.00680559896863997, 'sad': 10.03282368183136, 'surprise': 0.0020465062334551476, 'neutral': 86.23150587081909}, 'dominant_emotion': 'neutral', 'region': {'x': 30, 'y': 50, 'w': 65, 'h': 65}, 'face_confidence': 8.167705099273007}]
[{'emotion': {'angry': 0.4706279840320349, 'disgust': 5.386959975317573e-10, 'fear': 1.1910023167729378, 'happy': 0.000255436270890641, 'sad': 22.509941458702087, 'surprise': 5.6404114712194087e-08, 'neutral': 75.82817077636719}, 'dominant_emotion': 'neutral', 'region': {'x': 29, 'y': 51, 'w': 65, 'h'

##Download and results

In [ ]:
from google.colab import files
files.download('YearbookOutput.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>